<a href="https://colab.research.google.com/github/dowrave/Tensorflow_Basic/blob/main/220518_Advanced_tf_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

# tf.function으로 성능 향상하기
- 즉시 실행 : 직관적이고 유용하지만 성능, 배포에 비용이 더 든다 (단일 연산은 훨씬 간단하고 빠르다)
- 성능, 이식성을 생각한다면 `tf.function`을 써야 한다. 근데 이게 만병통치약은 아님.

## 여기서 배울 내용은 다음과 같다.
  - 즉시 실행 모드에서 디버깅 & `@tf.function`으로 데코리이팅
  - 객체 변경, 리스트 요소 추가 같은 파이썬 부수효과에 의존하는 것을 방지
  - `tf.function`은 텐서플로우 연산에 가장 잘 동작한다. 넘파이, 파이썬 호출은 `Constant`로 바뀜.

In [2]:
# 에러 출력을 위한 헬퍼 함수 정의
import traceback
import contextlib

@contextlib.contextmanager
def assert_raises(error_class):
  try:
    yield
  except error_class as e:
    print("예상된 예외 발생 \n : {}:".format(error_class))
    traceback.print_exc(limit = 2)
  except Exception as e:
    raise e
  else:
    raise Exception("{}를 기대했지만 에러 발생 없었음".format(error_class))

## 기초
- 기본 텐서플로우 연산과 동일함 (즉시 실행 모드 가능, 그래디언트 연산도 가능)
- `tf.function`은 즉시 실행보다 빠르다 : 특히 작은 연산이 많을 때 그러하며, 계산량이 많은 연산 몇 개로 이루어졌다면 속도 향상이 크지 않다(ex : Convolution)

In [3]:
import timeit
conv_layer = tf.keras.layers.Conv2D(100, 3)

@tf.function
def conv_fn(image):
  return conv_layer(image)

image = tf.zeros([1, 200, 200, 100])

# 워밍업 - 큰 차이 없음
conv_layer(image); conv_fn(image) 
print("즉시 실행 합성곱:", timeit.timeit(lambda: conv_layer(image), number=10)) # naive : 2.19 / GPU : 0.005
print("tf.function 합성곱:", timeit.timeit(lambda: conv_fn(image), number=10)) # naive : 2.38 /  GPU : 0.006

즉시 실행 합성곱: 0.005606202999999255
tf.function 합성곱: 0.006700272999992762


## 디버깅
- 즉시 실행모드가 디버깅하기 쉬움. 즉 즉시 실행을 하고 `tf.function`으로 데코리이팅할 것.
- 혹은 `tf.config.run_functions_eagerly(True)`로 전체 `tf.function`을 비활성화할 수도 있음.
<br>
[유의할 것]
<br>

  1. 파이썬 `print`는 트레이싱하는 동안에만 호출됨 -> 함수가 재트레이싱될 때 추적하는 데에 도움이 된다.
  2. `tf.print`는 언제나 실행됨 - 실행하는 동안 중간값 추적에 도움이 됨
  3. `tf.debugging.enable_check_numerics`은 쉽게 NaN과 Inf가 발생되는 곳을 추적할 수 있음
  4. `pdb`는 트레이싱이 일어나는 방식에 대한 도움을 줌 (`pdb`는 오토그래프가 변환한 소스코드를 보여줌)

## 트레이싱과 다형성
- 파이썬은 여러 종류의 매개변수 타입을 사용해 함수를 호출할 수 있고, 각기 다르게 수행함
- 텐서플로우 그래프는 `dtype`과 `shape`가 필요함. `tf.function`은 올바른 그래프를 생성하기 위해 필요하면 리트레이싱을 하는데, 여기서 대부분의 문제점이 옴
  - 리트레이싱 : (내 뇌피셜임) 다른 주소에 할당을 하는 게 아니라 같은 주소에 있는 정보를 갱신하는 방식
- 트레이싱이 중요한 이유 : 비싼 작업이라 그럼

In [12]:
# 함수와 다형성

@tf.function
def double(a):
  print("Tracing : ", a)
  return a + a

print(id(double(tf.constant(1)))) # int
print(double(tf.constant(1))) # int
print()
print(id(double(tf.constant(1.1)))) # float
print(double(tf.constant(1.1))) # float
print()
print(id(double(tf.constant('a')))) # string
print(double(tf.constant('a'))) # string
print()

Tracing :  Tensor("a:0", shape=(), dtype=int32)
139697042517648
tf.Tensor(2, shape=(), dtype=int32)

Tracing :  Tensor("a:0", shape=(), dtype=float32)
139697042517648
tf.Tensor(2.2, shape=(), dtype=float32)

Tracing :  Tensor("a:0", shape=(), dtype=string)
139697042517648
tf.Tensor(b'aa', shape=(), dtype=string)



- 위의 id를 보면 모두 같은 주소임

### 트레이싱 제어하기
- 새로운 `tf.function`을 만드는 방식.

In [14]:
def double(a):
  print("Tracing : ", a)
  return a + a


print(id(tf.function(double)(tf.constant(1)))) 
print(id(tf.function(double)(tf.constant(1.1))))
print(id(tf.function(double)(tf.constant("a"))))
# tf.function을 다르게 했더니 위의 모든 객체는 트레이싱이 따로 발생했음

Tracing :  Tensor("a:0", shape=(), dtype=int32)
139697022769872
Tracing :  Tensor("a:0", shape=(), dtype=float32)
139697022769680
Tracing :  Tensor("a:0", shape=(), dtype=string)
139697022769872


`get_concrete_function` 메소드 : 트레이싱된 특정 함수를 얻을 수 있다.

In [16]:
@tf.function
def double(a):
  print("Tracing : ", a)
  return a + a

# 콘크리트 함수 얻기
double_strings = double.get_concrete_function(tf.TensorSpec(shape = None, dtype = tf.string))

# 트레이싱 함수 실행
print(double_strings(tf.constant("a")))
print(double_strings(a = tf.constant("b")))

# 콘크리트 함수에 다른 타입을 사용하면 예외 발생
with assert_raises(tf.errors.InvalidArgumentError):
  double_strings(tf.constant(1))

Tracing :  Tensor("a:0", dtype=string)
tf.Tensor(b'aa', shape=(), dtype=string)
tf.Tensor(b'bb', shape=(), dtype=string)
예상된 예외 발생 
 : <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>:


Traceback (most recent call last):
  File "<ipython-input-2-5c8f6a317bb4>", line 8, in assert_raises
    yield
  File "<ipython-input-16-02f6023c4c8e>", line 16, in <module>
    double_strings(tf.constant(1))
tensorflow.python.framework.errors_impl.InvalidArgumentError: cannot compute __inference_double_261 as input #0(zero-based) was expected to be a string tensor but is a int32 tensor [Op:__inference_double_261]


`tf.function`에 `input_signature`를 지정해 트레이싱을 제한할 수 있다.

In [19]:
@tf.function(input_signature = (tf.TensorSpec(shape = [None], dtype = tf.int32),)) # 1차원의 int 데이터만 올 수 있게 제한한 듯?
def next_collatz(x):
  print("Tracing", x)
  return tf.where(x % 2 == 0, x // 2, 3  * x + 1)

print(next_collatz(tf.constant([1, 2])))

# input-signature에 1D Tensor를 지정했기 때문에 다음은 실패함
with assert_raises(ValueError):
  next_collatz(tf.constant([[1, 2], [3, 4]]))

Tracing Tensor("x:0", shape=(None,), dtype=int32)
tf.Tensor([4 1], shape=(2,), dtype=int32)
예상된 예외 발생 
 : <class 'ValueError'>:


Traceback (most recent call last):
  File "<ipython-input-2-5c8f6a317bb4>", line 8, in assert_raises
    yield
  File "<ipython-input-19-df7ade06db09>", line 10, in <module>
    next_collatz(tf.constant([[1, 2], [3, 4]]))
ValueError: Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32))
  input_signature: (
    TensorSpec(shape=(None,), dtype=tf.int32, name=None)).


## 리트레이싱이 언제 되나요?
- `tf.function`은 트레이싱으로 생성된 콘크리트 함수를 캐싱함
  - 캐시의 키는 함수의 위치 매개변수`arg`와 키워드 매개변수`kwargs`에서 생성된 키의 튜플이다.
  - `tf.Tensor` 매개변수를 위해 생성된 키는 차원 개수, 타입이 된다.
  - 파이썬 기본 자료형으로 생성된 키는 해당 변수의 값이 되며, 그 외의 타입에서 키는 `id()`를 기반으로 한다. 클래스 메소드는 인스턴스마다 독립적으로 트레이싱한다.
- 더 자세한 건 콘크리트 함수를 보세요

## 파이썬 매개변수 vs 텐서 매개변수
- 파이썬 매개변수 예시 : `num_layer = 10`, `training = True`, `nonlinearity = 'relu'` 등
- 파이썬 매개변수가 그래프에 사용되지 않는 경우가 있고, 이런 경우 파이썬 값이 변하면 불필요한 리트레이싱이 발생함.
<br>
- 다음 예제는 오토그래프가 동적으로 펼치는 반복 루프이다. 다중 트레이싱이지만 생성된 그래프는 실제로 동일해서 비효율적임

In [22]:
import time

In [25]:
def train_one_step():
  pass

@tf.function
def train(num_steps):
  print("트레이싱 num_steps = {}".format(num_steps))

  for _ in tf.range(num_steps):
    train_one_step()
start = time.time()
train(num_steps = 10)
train(num_steps = 20)
print(time.time() - start)

트레이싱 num_steps = 10
트레이싱 num_steps = 20
0.1270909309387207


- 위의 리트레이싱을 해결하는 간단한 방법은 매개변수를 `Tensor`로 바꾸는 것이다. (속도에 관한 차이는 별로 보이지 않음)

In [26]:
train(num_steps = tf.constant(10))
train(num_steps = tf.constant(20))

트레이싱 num_steps = Tensor("num_steps:0", shape=(), dtype=int32)
0.11896872520446777


## tf.function의 부수 효과
- 파이썬 부수 효과만을 이용해 트레이싱을 디버깅한다. 
- 기본 : 파이썬 코드는 호출횟수와 실행횟수가 동일하지 않음. 트레이싱 때만 실행됨

In [27]:
@tf.function
def f(x):
  print("트레이싱", x) 
  tf.print("실행", x)

f(1)
f(1)
f(2)

트레이싱 1
실행 1
실행 1
트레이싱 2
실행 2


`tf.function`을 호출할 때마다 파이썬 코드를 실행하려면 `tf.py_function`을 써야 한다.
- 단점 : 이식성, 성능이 좋지 않고 분산환경에서 잘 동작하지 않는다.
  - 또한 `tf.py_function`은 미분가능하도록 그래프를 만들기 때문에 모든 입출력을 텐서로 전환함

In [28]:
external_list = []

def side_effect(x):
  print("파이썬 부수 효과")
  external_list.append(x)

@tf.function
def f(x):
  tf.py_function(side_effect, inp = [x], Tout = [])

f(1)
f(1)
f(1)
assert len(external_list) == 3

# py_function이 1을 tf.constant(1)로 바꾸므로 .numpy()를 호출한다.
assert external_list[0].numpy() == 1

파이썬 부수 효과
파이썬 부수 효과
파이썬 부수 효과


## 파이썬 상태 주의하기
- 제너레이터, 반복자 같은 파이썬의 기능은 파이썬 런타임에 의존한다.
- 일반적으로 즉시 실행에선 동일하게 동작하지만 `tf.function` 안에서는 예상 밖의 일이 일어날 수 있다.

In [30]:
external_var = tf.Variable(0)

@tf.function
def gr_buggy_consume_next(iterator):
  external_var.assign_add(next(iterator))
  tf.print("external_var의 값 : ", external_var)

def py_buggy_consume_next(iterator):
  external_var.assign_add(next(iterator))
  tf.print("external_var의 값 : ", external_var)

iterator = iter([0, 1, 2, 3])
# 다음 반복자를 얻는 건 파이썬의 효과임 - 트레이싱에서만 진행됨
gr_buggy_consume_next(iterator)
gr_buggy_consume_next(iterator)
gr_buggy_consume_next(iterator)

# 파이썬 작업이므로 계속 실행됨
py_buggy_consume_next(iterator)
py_buggy_consume_next(iterator)
py_buggy_consume_next(iterator)

external_var의 값 :  0
external_var의 값 :  0
external_var의 값 :  0
external_var의 값 :  1
external_var의 값 :  3
external_var의 값 :  6


## 변수
- 변수는 즉시 실행 모드와 그래프 모드에서 서로 다르게 동작하는 코드를 만들 수 있다.
- 특히 호출마다 새로운 변수를 만들 때 일어남. 
  - `tf.function`은 호출마다 같은 변수를 재사용한다. 
  - 즉시 실행 모드에서는 호출마다 새로운 변수가 생성됨.
  - 함수 내에 `tf.Variable`을 만들면 `Variable`을 위 2개 중 어느 것으로 할 지 모호하기 때문에 에러를 반환한다.
    - `@tf.function`을 씌우든 아니든 둘 모두 에러가 발생하는 거 확인했음

In [35]:
@tf.function
def f(x):
  v = tf.Variable(1.)
  v.assign_add(x)
  return v

with assert_raises(ValueError):
  f(1.)

예상된 예외 발생 
 : <class 'ValueError'>:


Traceback (most recent call last):
  File "<ipython-input-2-5c8f6a317bb4>", line 8, in assert_raises
    yield
  File "<ipython-input-35-e5c35e03649a>", line 8, in <module>
    f(1.)
ValueError: in user code:

    File "<ipython-input-35-e5c35e03649a>", line 3, in f  *
        v = tf.Variable(1.)

    ValueError: tf.function only supports singleton tf.Variables created on the first call. Make sure the tf.Variable is only created once or created outside tf.function. See https://www.tensorflow.org/guide/function#creating_tfvariables for more information.



- 아래 코드는 모호하지 않아서 잘 작동함. (함수 밖에서 변수가 선언되었음)

In [32]:
v = tf.Variable(1.)

@tf.function
def f(x):
  return v.assign_add(x)

print(f(1.))
print(f(2.))

tf.Tensor(2.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


- 혹은 함수가 처음 호출될 때만 변수가 생성되도록 설정하는 방법도 있음

In [36]:
class C:
  pass

obj = C()
obj.v = None

@tf.function
def g(x):
  if obj.v is None:
    obj.v = tf.Variable(1.)
  return obj.v.assign_add(x)

print(g(1.))
print(g(2.))

tf.Tensor(2.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


- 변수 초기화가 함수 매개변수와 다른 변수 값에 의존할 수 있음. 올바른 초기화 순서를 찾기 위해 제어 의존성을 생성하는 메소드를 쓸 수도 있다.


In [37]:
state = []

@tf.function
def fn(x):
  if not state:
    state.append(tf.Variable(2. * x))
    state.append(tf.Variable(state[0] * 3.))

  return state[0] * x + state [1]

print(fn(tf.constant(1.)))
print(fn(tf.constant(3.)))


tf.Tensor(8.0, shape=(), dtype=float32)
tf.Tensor(12.0, shape=(), dtype=float32)


## 오토그래프 변환
- `tf.function` 내에 기본으로 활성화 되어 있음.
- 파이썬의 즉시 실행 코들르 그래프 호환 텐서플로우로 변환함
  - 여기에는 `if`, `for`, `while`문이 포함됨.
- `tf.cond`와 `tf.while_loop` 등이 사용가능하지만 제어 흐름은 파이썬으로 작성하는 게 만들기도 이해하기도 쉽다.

In [38]:
@tf.function
def f(x):
  while tf.reduce_sum(x) > 1:
    tf.print(x)
    x = tf.tanh(x)
  return x

f(tf.random.uniform([5]))

[0.519893527 0.782256484 0.780321121 0.809718251 0.249633908]
[0.47761786 0.654 0.65289104 0.669434845 0.244574472]
[0.444333792 0.574356616 0.573613 0.584608078 0.239811838]
[0.417230278 0.518551648 0.518007696 0.526006639 0.235317975]
[0.394594491 0.476581395 0.476160854 0.482322633 0.231068507]
[0.375314265 0.443501592 0.443163663 0.448101848 0.227042019]
[0.358631283 0.416542679 0.416263342 0.420337498 0.223219618]
[0.34400785 0.394013852 0.393777817 0.397214711 0.219584569]
[0.331050694 0.374815315 0.374612451 0.377563238 0.216122046]
[0.319464535 0.358196408 0.35801959 0.360589385 0.212818801]
[0.309022635 0.343624383 0.343468428 0.345733076 0.209662974]
[0.299547672 0.330709249 0.33057031 0.332586 0.206643879]
[0.290898591 0.319157928 0.319033086 0.320842475 0.203751892]
[0.282961667 0.308745295 0.308632344 0.310268492 0.200978354]
[0.275643915 0.299295187 0.299192369 0.300681293 0.198315352]
[0.268868625 0.290667474 0.290573299 0.291935921 0.195755735]
[0.262571782 0.282749027 

<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([0.20179412, 0.21050249, 0.21046686, 0.2109808 , 0.16485177],
      dtype=float32)>

### 오토그래프가 생성한 코드 확인

In [39]:
print(tf.autograph.to_code(f.python_function))

def tf__f(x):
    with ag__.FunctionScope('f', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return (x,)

        def set_state(vars_):
            nonlocal x
            (x,) = vars_

        def loop_body():
            nonlocal x
            ag__.converted_call(ag__.ld(tf).print, (ag__.ld(x),), None, fscope)
            x = ag__.converted_call(ag__.ld(tf).tanh, (ag__.ld(x),), None, fscope)

        def loop_test():
            return (ag__.converted_call(ag__.ld(tf).reduce_sum, (ag__.ld(x),), None, fscope) > 1)
        ag__.while_stmt(loop_test, loop_body, get_state, set_state, ('x',), {})
        try:
            do_return = True
            retval_ = ag__.ld(x)
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)



### 조건문
- `if <condition>` 문장을 `tf.cond` 호출로 변경한다. `<condition>`이 텐서일 때 수행되며 아니라면 `if` 문은 파이썬 조건문으로 실행됨.
- 트레이싱하는 동안 파이썬 조건문을 실행하므로 정확히 하나의 조건 분기만 그래프에 추가되며, 오토그래프가 없다면 트레이싱된 그래프는 제어흐름을 바꿀 수 없다.
- `tf.cond`는 조건 분기를 트레이싱하고 그래프에 추가, 실행 시 동적으로 분기를 선택한다.

In [41]:
@tf.function
def fizzbuzz(n):
  for i in tf.range(1, n + 1):
    print('루프 트레이싱')
    if i % 15 == 0:
      print('fizzbuzz 브랜치 트레이싱')
      tf.print('fizzbuzz')
    elif i % 3 == 0:
      print('fizz 브랜치 트레이싱')
      tf.print('fizz')
    elif i % 5 == 0:
      print('buzz 브랜치 트레이싱')
      tf.print('buzz')
    else:
      print('디폴트 브랜치 트레이싱')
      tf.print(i)

fizzbuzz(tf.constant(5))
tf.print()
fizzbuzz(tf.constant(20))

루프 트레이싱
fizzbuzz 브랜치 트레이싱
fizz 브랜치 트레이싱
buzz 브랜치 트레이싱
디폴트 브랜치 트레이싱
1
2
fizz
4
buzz

1
2
fizz
4
buzz
fizz
7
8
fizz
buzz
11
fizz
13
14
fizzbuzz
16
17
fizz
19
buzz


- 트레이싱을 한번 쫙 하고 실행되는 걸 볼 수 있다.

### 반복문
- 일부 `for`, `while`문을 `tf.while_loop`와 같은 동등한 텐서플로우 반복 연산으로 바꾼다. 바뀌지 않는다면 파이썬으로 실행됨.
  - `for x in y` - y가 텐서면 `tf.while_loop`로 변환된다. y가 `tf.data.Dataset`이라면 `tf.data.Dataset` 연산의 조합이 생성된다.
  - `while <condition>` : `<condition>`이 텐서면 `tf.while_loop`로 변환된다.
- 텐서플로우는 반복문 블럭을 트레이싱해 반복의 수행을 동적으로 선택한다. 반복문 블럭은 `tf.Graph`에 1번만 포함된다.

#### 파이썬 데이터로 반복하기
- 흔한 실수 : `tf.function` 안에서 파이썬, 넘파이 데이터로 반복하는 것 -> 트레이싱 과정에서 반복이 추가되므로 `tf.Graph`에 복사된 모델이 추가되어 버린다.
- `tf.function`으로 반복을 감싸고 싶다면 데이터를 `tf.data.Dataset`으로 감싸는 것이다. - 오토그래프가 동적으로 훈련 반복을 펼친다.

#### from_generator와 from_tensors의 차이
- 전자 : 파이썬에서 데이터 유지, `tf.py_function`으로 데이터를 가져옴 -> 성능에 영향이 감
- 후자 : 그래프에 하나의 큰 `tf.constant()` 노드로 데이터를 복사하므로 메모리에 영향이 감

### 가장 효율적인 데이터 소비 방법
`TFRecordDataset`, `CsvDataset` 등으로 파일에서 데이터를 읽는 것이 가장 효율적이다. 파이썬을 거치지 않고 비동기적으로 데이터를 적재하고 프리페칭할 수 있기 때문이다. tf.data 가이드를 참고할 것.
